In [1]:
!pip install pyecharts

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.7/147.7 kB 885.6 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.6/143.6 kB 678.9 kB/s eta 0:00:000:0100:01m


In [2]:
import requests
import json
from urllib.parse import urlparse
from urllib.parse import parse_qs
import pandas

def call_sg_api(resource_id, limit=None, params=None, records_total=None):
    
    def go_to_next_url(next_url):
        parsed_url = urlparse(next_url)
        offset = parse_qs(parsed_url.query)['offset'][0]
        params['offset'] = offset
        params.pop('resource_id')
        return call_sg_api(resource_id, limit, params, records_total)

    params = params if params else {}
    records_total = records_total if records_total else []
    url = 'https://data.gov.sg/api/action/datastore_search'
    
    params['limit'] = 10000
    params = {'resource_id': resource_id, **params}
    result = requests.get(url=url, params=params).json()
    new_records = result['result']['records']
    
    if len(new_records) == 0:
        return pandas.DataFrame.from_records(records_total)
    
    records_total += new_records
    
    if limit and len(records_total) > limit:
        records_total = records_total[:limit]
        return pandas.DataFrame.from_records(records_total)

    next_url = result['result']['_links']['next']
    return go_to_next_url(next_url)

In [3]:
resource_id = 'dec53407-9f97-47b8-ba89-b2070569a09e'
df_raw_data = call_sg_api(resource_id=resource_id)

In [4]:
df_raw_data

,_id,year,category,type,number
0,1,2005,Cars and Station-wagons,Private cars,401638
1,2,2006,Cars and Station-wagons,Private cars,421904
2,3,2007,Cars and Station-wagons,Private cars,451745
3,4,2008,Cars and Station-wagons,Private cars,476634
4,5,2009,Cars and Station-wagons,Private cars,497116
...,...,...,...,...,...
323,324,2020,Buses,Excursion buses,7779
324,325,2020,Tax Exempted Vehicles,Cars and Station-wagons,2138
325,326,2020,Tax Exempted Vehicles,Motorcycles and scooters,621
326,327,2020,Tax Exempted Vehicles,Buses,339
